In [1]:
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

Read the CSV and Perform Basic Data Cleaning

In [50]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


Select your features (columns)¶

In [52]:
X = df.drop('koi_disposition',axis=1)
y = df['koi_disposition']
print(X.shape, y.shape)

(6991, 40) (6991,)


In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [54]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,7.380000e-05,-7.380000e-05,133.077240,0.008440,-0.008440,...,-171,4.327,0.153,-0.187,1.125,0.310,-0.207,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,6.060000e-06,-6.060000e-06,132.020050,0.007950,-0.007950,...,-175,4.578,0.033,-0.187,0.797,0.211,-0.056,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,6.540000e-05,-6.540000e-05,134.460380,0.006190,-0.006190,...,-189,4.481,0.050,-0.200,0.963,0.290,-0.097,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,1.910000e-05,-1.910000e-05,174.662240,0.001820,-0.001820,...,-85,4.536,0.056,-0.016,0.779,0.023,-0.049,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,5.150000e-07,-5.150000e-07,172.258529,0.000083,-0.000083,...,-77,4.359,0.110,-0.110,1.082,0.173,-0.130,292.16705,48.727589,15.263


Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [55]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X_train)

In [56]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [57]:
from tensorflow.keras.utils import to_categorical

In [60]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [62]:
y_train=le.transform(y_train)
y_test=le.transform(y_test)

In [63]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

Create / Train Neural Net Model

In [74]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [75]:
from tensorflow.keras.layers import Dense
number_inputs = 40
number_hidden_nodes = 15
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_hidden_nodes))

In [80]:
number_classes = 3
model.add(Dense(units=number_classes, activation='softmax'))

In [81]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 15)                615       
_________________________________________________________________
dense_19 (Dense)             (None, 15)                240       
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 80        
_________________________________________________________________
dense_21 (Dense)             (None, 3)                 18        
Total params: 935
Trainable params: 935
Non-trainable params: 0
_________________________________________________________________


In [82]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=200,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/200
5243/5243 - 2s - loss: 1.0441 - accuracy: 0.4507
Epoch 2/200
5243/5243 - 1s - loss: 0.7871 - accuracy: 0.7059
Epoch 3/200
5243/5243 - 1s - loss: 0.5933 - accuracy: 0.7494
Epoch 4/200
5243/5243 - 1s - loss: 0.5307 - accuracy: 0.7496
Epoch 5/200
5243/5243 - 1s - loss: 0.4970 - accuracy: 0.7496
Epoch 6/200
5243/5243 - 1s - loss: 0.4752 - accuracy: 0.7494
Epoch 7/200
5243/5243 - 1s - loss: 0.4599 - accuracy: 0.7494
Epoch 8/200
5243/5243 - 1s - loss: 0.4477 - accuracy: 0.7494
Epoch 9/200
5243/5243 - 1s - loss: 0.4390 - accuracy: 0.7496
Epoch 10/200
5243/5243 - 1s - loss: 0.4324 - accuracy: 0.7494
Epoch 11/200
5243/5243 - 1s - loss: 0.4261 - accuracy: 0.7496
Epoch 12/200
5243/5243 - 1s - loss: 0.4209 - accuracy: 0.7461
Epoch 13/200
5243/5243 - 1s - loss: 0.4173 - accuracy: 0.7496
Epoch 14/200
5243/5243 - 1s - loss: 0.4146 - accuracy: 0.7494
Epoch 15/200
5243/5243 - 1s - loss: 0.4104 - accuracy: 0.7442
Epoch 16/200
5243/5243 - 1s - loss: 0.4087 - accuracy: 0.

Epoch 133/200
5243/5243 - 1s - loss: 0.2638 - accuracy: 0.8945
Epoch 134/200
5243/5243 - 1s - loss: 0.2642 - accuracy: 0.8899
Epoch 135/200
5243/5243 - 1s - loss: 0.2659 - accuracy: 0.8879
Epoch 136/200
5243/5243 - 1s - loss: 0.2630 - accuracy: 0.8915
Epoch 137/200
5243/5243 - 1s - loss: 0.2707 - accuracy: 0.8880
Epoch 138/200
5243/5243 - 1s - loss: 0.2589 - accuracy: 0.8961
Epoch 139/200
5243/5243 - 1s - loss: 0.2636 - accuracy: 0.8920
Epoch 140/200
5243/5243 - 1s - loss: 0.2656 - accuracy: 0.8879
Epoch 141/200
5243/5243 - 1s - loss: 0.2604 - accuracy: 0.8917
Epoch 142/200
5243/5243 - 1s - loss: 0.2640 - accuracy: 0.8888
Epoch 143/200
5243/5243 - 1s - loss: 0.2603 - accuracy: 0.8919
Epoch 144/200
5243/5243 - 1s - loss: 0.2641 - accuracy: 0.8915
Epoch 145/200
5243/5243 - 1s - loss: 0.2599 - accuracy: 0.8959
Epoch 146/200
5243/5243 - 1s - loss: 0.2652 - accuracy: 0.8854
Epoch 147/200
5243/5243 - 1s - loss: 0.2602 - accuracy: 0.8919
Epoch 148/200
5243/5243 - 1s - loss: 0.2603 - accuracy:

In [84]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1 - 1s - loss: 0.3940 - accuracy: 0.8822
Loss: 0.28014800409430496, Accuracy: 0.8821510076522827
